In [ ]:
!pip install nltk

In [ ]:
import nltk
import os
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import numpy as np
from gensim.models import Nmf
from gensim.models.ldamodel import LdaModel
from gensim.models import LsiModel
from gensim.models import HdpModel
from gensim.models import LdaMulticore
from gensim.models import EnsembleLda
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import gensim
import gensim.corpora as corpora
from gensim.models import Phrases
from gensim.utils import simple_preprocess
from gensim.corpora.dictionary import Dictionary
from gensim.utils import tokenize
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
import re
import json
import string
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import (
    strip_non_alphanum,
    split_alphanum,
    strip_short,
    strip_numeric
)

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import seaborn as sns

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
datafile = 'latest_ticket_data.csv'
data = pd.read_csv(datafile)

In [ ]:
data['Description'] = data['Description'].str.lower()
data['Description'] = data['Description'].astype(str)

In [ ]:
stop_words = stopwords.words('english')
custom_stop_words = ['hi', 'since', 'please', 'best', 'regards', 'thank', 'thanks', 'hello', 'sent', 'great', 'dear', 'help', 'kind']
time_words = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'today' , 'yesterday', 'tomorrow', 'hour', 'hours', 'time', 'times', 'timelines', 'date', 'day', 'days', 'am', 'pm', 'morning', 'noon', 'afternoon', 'evening', 'night', 'winter', 'summer', 'rain', 'cold']

def remove_stop_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_custom_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(custom_stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_time_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(time_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

data['Description'] = data["Description"].map(lambda x: remove_stop_words(x))
data['Description'] = data["Description"].map(lambda x: remove_custom_words(x))
data['Description'] = data["Description"].map(lambda x: remove_time_words(x))

In [ ]:
def parse_html(text, parser="html.parser"):
    soup = BeautifulSoup(text, parser)
    soup = remove_html_tags(soup)
    text = remove_multiple_space(soup.get_text()).strip()
    return text


def parse_html_v2(text, loop=2, parser="html.parser"):
    if not text:
        text = ""
    # some contents still have html code after first parse
    # loop solved problem
    for _ in range(loop):
        soup = BeautifulSoup(text, parser)
        text = soup.get_text()
    text = remove_multiple_space(text)
    return text


def remove_links_content(text):
    text = re.sub(r"http\S+", "", text)
    return text


def remove_emails(text):
    return re.sub('\S*@\S*\s?', '', text)  # noqa


def remove_punctuation(text):
    """https://stackoverflow.com/a/37221663"""
    table = str.maketrans({key: None for key in string.punctuation})
    return text.translate(table)


def remove_special_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('{.*?}')
    return re.sub(clean, '', text)


def preprocess_text(text):
    text = parse_html_v2(text)
    text = text.lower()
    text = remove_links_content(text)
    text = remove_emails(text)
    text = remove_special_tags(text)  # remove content between {}
    text = remove_punctuation(text)  # remove all puntuations
    text = split_alphanum(text)  # add space between word and numeric
    text = strip_numeric(text)  # remove digits
    text = strip_non_alphanum(text)  # remove non-alphabetic characters
    text = strip_short(text, minsize=2)  # remove word with length  <  minsize
    text = remove_multiple_space(text).strip()  # remove space and strip
    #text = tokenize(text)
    return text


def remove_multiple_space(text):
    return re.sub("\s\s+", " ", text)  # noqa


def remove_html_tags(soup,
                     tags=["script", "style"],
                     get_text=False):
    for tag in tags:
        for sample in soup.find_all(tag):
            sample.replaceWith('')

    if get_text:
        return soup.get_text()
    return soup

In [ ]:
# Convert body column to string for performing text operations
data['Description'] = data['Description'].astype(str)
data['Description'] =data["Description"].map(lambda x: preprocess_text(x))

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


x = data['Description'].values.tolist()
docs = list(sent_to_words(x))
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

In [ ]:
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [ ]:
#Remove rare & common tokens
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.2)
#Create dictionary and corpus required for Topic Modeling
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))
print(corpus[:1])

Number of unique tokens: 1365
Number of documents: 3000
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [ ]:
# Set parameters.
num_topics = 5
chunksize = 500
passes = 20
iterations = 400
eval_every = 1

# Make a index to word dictionary.
temp = dictionary[0]  # only to "load" the dictionary.
id2word = dictionary.id2token

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
data['Label'] = LE.fit_transform(data['Category'])

In [ ]:
from sklearn.metrics import accuracy_score
def getModelAccuacy(model, corpus, dataset, Label):
  get_document_topics=model.__getitem__(corpus)
  #np_array_of_objects = np.asarray(get_document_topics, dtype="object")
  pred_df = pd.DataFrame(columns=['pred_label'])
  length = len(get_document_topics)
  dict_list = []
  for i in range(0,length):
    count=len(get_document_topics[i])
    max=0
    for j in range(0,count):
      if(max < get_document_topics[i][j][1]):
        max=get_document_topics[i][j][1]
        label=get_document_topics[i][j][0]
    row_dict = {'pred_label': label}
    dict_list.append(row_dict)
  pred_df = pd.DataFrame.from_dict(dict_list)
  y_pred = pred_df['pred_label'].values
  y_true = dataset['Label'].values
  return(accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
def getLDAModelAccuacy(model, corpus, dataset, Label):
  get_document_topics = [model.get_document_topics(item) for item in corpus]
  np_array_of_objects = np.asarray(get_document_topics, dtype="object")
  pred_df = pd.DataFrame(columns=['pred_label'])
  length = len(pd.DataFrame(np_array_of_objects)[0])
  dict_list = []
  for i in range(0,length):
    count=len(pd.DataFrame(np_array_of_objects)[0][i])
    max=0
    for j in range(0,count):
      if(max < (pd.DataFrame(np_array_of_objects)[0][i])[j][1]):
        max=(pd.DataFrame(np_array_of_objects)[0][i])[j][1]
        label=(pd.DataFrame(np_array_of_objects)[0][i])[j][0]
    row_dict = {'pred_label': label}
    dict_list.append(row_dict)
  pred_df = pd.DataFrame.from_dict(dict_list)
  y_pred = pred_df['pred_label'].values
  y_true = dataset['Label'].values
  return(accuracy_score(y_true, y_pred))

In [ ]:
#nmf model
nmf_model = Nmf(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       w_max_iter=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 5 topics
print(nmf_model.print_topics())

[(0, '0.099*"leaver" + 0.038*"leaver_form" + 0.034*"leaver_leaver" + 0.032*"approver" + 0.021*"approver_approver" + 0.018*"mobile_phone" + 0.017*"form" + 0.017*"location" + 0.017*"mailbox" + 0.016*"personal_mailbox"'), (1, '0.136*"user" + 0.046*"specialist" + 0.044*"application" + 0.035*"administrator" + 0.031*"setup" + 0.019*"role" + 0.014*"assigned" + 0.012*"contract" + 0.011*"information" + 0.011*"name"'), (2, '0.170*"changed" + 0.169*"interface" + 0.101*"link" + 0.069*"switch" + 0.025*"port" + 0.022*"event" + 0.018*"change" + 0.018*"active" + 0.015*"issues" + 0.014*"consider_environment"'), (3, '0.061*"si" + 0.042*"senior_engineer" + 0.037*"la" + 0.034*"item" + 0.032*"ca" + 0.029*"care" + 0.022*"engineer" + 0.020*"added" + 0.020*"se" + 0.020*"visitor"'), (4, '0.028*"cards" + 0.022*"access" + 0.017*"floor" + 0.015*"card" + 0.014*"visitor" + 0.014*"analyst_ext" + 0.014*"visitor_cards" + 0.012*"required" + 0.012*"permanent" + 0.010*"ground_floor"')]


In [ ]:
getLDAModelAccuacy(nmf_model,corpus,data,"Label")

0.20433333333333334

In [ ]:
c_v_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_nmf = c_v_coherence_model_nmf.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_nmf)


C_V Coherence Score:  0.5316297291547925


In [ ]:
u_mass_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_nmf = u_mass_coherence_model_nmf.get_coherence()
print('\nU_Mass Coherence Score: ', u_mass_coherence_nmf)


U_Mass Coherence Score:  -3.338684001385441


In [ ]:
c_uci_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_nmf = c_uci_coherence_model_nmf.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_nmf)


c_uci Coherence Score:  -5.430208599542743


In [ ]:
c_npmi_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_nmf = c_npmi_coherence_model_nmf.get_coherence()
print('\c_npmi_Coherence Score: ', c_npmi_coherence_nmf)

\c_npmi_Coherence Score:  -0.11740587490771678


In [ ]:
#lda model
lda_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 5 topics
print(lda_model.print_topics())

[(0, '0.017*"importance_high" + 0.016*"engineer" + 0.015*"senior_engineer" + 0.010*"senior" + 0.009*"also" + 0.009*"log" + 0.008*"working" + 0.008*"client" + 0.008*"update" + 0.008*"let"'), (1, '0.034*"disk" + 0.026*"name" + 0.024*"purchase_po" + 0.022*"item" + 0.021*"receive_item" + 0.020*"secure_area" + 0.017*"host" + 0.016*"design_lead" + 0.016*"printer" + 0.015*"changed"'), (2, '0.075*"card" + 0.029*"floor" + 0.029*"la" + 0.028*"access_card" + 0.028*"lost" + 0.026*"si" + 0.025*"cards" + 0.025*"visitor" + 0.024*"lost_access" + 0.022*"access"'), (3, '0.023*"th_floor" + 0.021*"event" + 0.021*"access" + 0.020*"analyst_ext" + 0.019*"could" + 0.018*"let_know" + 0.017*"en" + 0.015*"details" + 0.014*"issue" + 0.013*"laptop"'), (4, '0.055*"leaver" + 0.046*"form" + 0.035*"senior_developer" + 0.032*"find_attached" + 0.025*"starter_form" + 0.024*"leaver_form" + 0.024*"room" + 0.023*"maternity_leaver" + 0.022*"attached" + 0.021*"could_investigate"')]


In [ ]:
getLDAModelAccuacy(lda_model,corpus,data,"Label")

0.292

In [ ]:
c_v_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_lda = c_v_coherence_model_lda.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_lda)


C_V Coherence Score:  0.43983831271107104


In [ ]:
u_mass_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_lda = u_mass_coherence_model_lda.get_coherence()
print('\nU_Mass Coherence Score: ', u_mass_coherence_lda)


U_Mass Coherence Score:  -6.9365875149635645


In [ ]:
c_uci_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_lda = c_uci_coherence_model_lda.get_coherence()
print('\nc_uciCoherence Score: ', c_uci_coherence_lda)


c_uciCoherence Score:  -5.0454374515094536


In [ ]:
c_npmi_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_lda = c_npmi_coherence_model_lda.get_coherence()
print('\c_npmi_Coherence Score: ', c_npmi_coherence_lda)

\c_npmi_Coherence Score:  -0.11965968630081467


In [ ]:
lsi_model = LsiModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       num_topics=num_topics)
# Print the Keyword in the 5 topics
print(lsi_model.print_topics())

[(0, '0.615*"changed" + 0.614*"interface" + 0.367*"link" + 0.250*"switch" + 0.090*"port" + 0.083*"event" + 0.066*"active" + 0.062*"change" + 0.057*"senior_engineer" + 0.055*"issues"'), (1, '0.414*"visitor" + 0.292*"cards" + 0.266*"visitor_visitor" + 0.258*"analyst_ext" + 0.252*"access" + 0.187*"visitor_cards" + 0.180*"si" + 0.175*"floor" + 0.167*"card" + 0.135*"required"'), (2, '0.324*"visitor" + -0.234*"item" + -0.227*"user" + -0.222*"log" + 0.219*"visitor_visitor" + -0.183*"event" + 0.178*"cards" + 0.148*"visitor_cards" + -0.145*"failed" + -0.145*"disk"'), (3, '0.609*"kb" + 0.479*"update" + 0.338*"based" + 0.228*"edition" + 0.228*"bit" + 0.157*"event" + -0.123*"log" + 0.108*"disk" + -0.086*"job" + -0.086*"item"'), (4, '-0.384*"event" + -0.298*"disk" + 0.275*"kb" + 0.233*"log" + -0.223*"indicated" + 0.194*"update" + -0.187*"problem" + 0.180*"job" + 0.175*"failed" + -0.168*"replace"')]


In [ ]:
getModelAccuacy(lsi_model,corpus,data,"Label")

0.19033333333333333

In [ ]:
c_v_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_lsi = c_v_coherence_model_lsi.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_lsi)


C_V Coherence Score:  0.4056345017867004


In [ ]:
u_mass_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_lsi = u_mass_coherence_model_lsi.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_lsi)


u_mass Coherence Score:  -6.3711717709480755


In [ ]:
c_uci_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_lsi = c_uci_coherence_model_lsi.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_lsi)


c_uci Coherence Score:  -6.79842747737292


In [ ]:
c_npmi_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_lsi = c_npmi_coherence_model_lsi.get_coherence()
print('\nc_npmi Coherence Score: ', c_npmi_coherence_lsi)


c_npmi Coherence Score:  -0.17749792919846552


In [ ]:
#hdp model
hdp_model = HdpModel(corpus=corpus, id2word=id2word, chunksize=chunksize)
# Print the Keyword in the 5 topics
print(hdp_model.print_topics())

[(0, '0.005*assign + 0.004*liaise + 0.004*wanted + 0.004*confidentiality + 0.004*event_severity + 0.003*assigning + 0.003*used + 0.003*logged + 0.003*ext + 0.003*following'), (1, '0.005*previous_status + 0.005*secured + 0.005*period + 0.005*control + 0.004*cards + 0.004*addressee + 0.004*assigned_advised_record_number + 0.004*sending + 0.004*certificate + 0.004*analyst_ext'), (2, '0.006*feedback + 0.006*removed + 0.005*importance + 0.005*writing + 0.005*working_properly + 0.005*current + 0.004*safe + 0.004*individual + 0.004*says + 0.004*nothing'), (3, '0.006*rule + 0.004*host + 0.004*apologies + 0.004*even + 0.004*resource + 0.004*trouble + 0.003*result + 0.003*storage_required + 0.003*fie + 0.003*expected'), (4, '0.005*reboot + 0.004*ti + 0.004*confirm + 0.004*assistants + 0.004*highlighted + 0.004*owner + 0.003*hosted + 0.003*event_code + 0.003*virtual + 0.003*el'), (5, '0.005*issues + 0.004*take_look + 0.004*visitor_cards + 0.004*high_priority + 0.004*document + 0.004*raise_ticket 

In [ ]:
getModelAccuacy(hdp_model,corpus,data,"Label")

0.004666666666666667

In [ ]:
c_v_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_hdp = c_v_coherence_model_hdp.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_hdp)


C_V Coherence Score:  0.6385758401321195


In [ ]:
u_mass_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_hdp = u_mass_coherence_model_hdp.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_hdp)


u_mass Coherence Score:  -16.3361682120392


In [ ]:
c_uci_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_hdp = c_uci_coherence_model_hdp.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_hdp)


c_uci Coherence Score:  -13.552392017212053


In [ ]:
c_npmi_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_hdp = c_npmi_coherence_model_hdp.get_coherence()

print('\nc_npmi Coherence Score: ', c_npmi_coherence_hdp)


c_npmi Coherence Score:  -0.47886777780505546


In [ ]:
#lda multicore model
lda_multicore_model = LdaMulticore(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 5 topics
print(lda_multicore_model.print_topics())

[(0, '0.018*"importance_high" + 0.012*"issue" + 0.010*"could" + 0.010*"access" + 0.010*"en" + 0.010*"th_floor" + 0.010*"working" + 0.009*"floor" + 0.009*"let_know" + 0.009*"laptop"'), (1, '0.022*"leaver" + 0.021*"name" + 0.014*"meeting_room" + 0.014*"find_attached" + 0.013*"form" + 0.012*"leave" + 0.012*"information" + 0.011*"annual_leave" + 0.010*"leaver_form" + 0.009*"floor"'), (2, '0.038*"card" + 0.033*"event" + 0.023*"lost" + 0.023*"access_card" + 0.022*"disk" + 0.020*"access" + 0.019*"lost_access" + 0.016*"details" + 0.013*"original_message" + 0.013*"changed"'), (3, '0.029*"purchase_po" + 0.026*"receive_item" + 0.024*"engineer_ext" + 0.019*"upgrade" + 0.019*"item" + 0.018*"administrator" + 0.016*"log" + 0.016*"receive" + 0.016*"log_allocation" + 0.015*"po"'), (4, '0.047*"card" + 0.044*"la" + 0.043*"si" + 0.034*"senior_engineer" + 0.031*"visitor" + 0.030*"va" + 0.027*"cards" + 0.024*"ca" + 0.021*"visitor_cards" + 0.020*"care"')]


In [ ]:
getLDAModelAccuacy(lda_multicore_model,corpus,data,"Label")

0.17933333333333334

In [ ]:
getModelAccuacy_2(lda_multicore_model,corpus,data,"Label")

0.179

In [ ]:
c_v_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_lda_multicore = c_v_coherence_model_lda_multicore.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_lda_multicore)


C_V Coherence Score:  0.4618258381260487


In [ ]:
u_mass_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_lda_multicore = u_mass_coherence_model_lda_multicore.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_lda_multicore)


u_mass Coherence Score:  -5.7525765893386875


In [ ]:
c_uci_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_lda_multicore = c_uci_coherence_model_lda_multicore.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_lda_multicore)


c_uci Coherence Score:  -4.618530579694157


In [ ]:
c_npmi_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_lda_multicore = c_npmi_coherence_model_lda_multicore.get_coherence()
print('\nc_npmi Coherence Score: ', c_npmi_coherence_lda_multicore)


c_npmi Coherence Score:  -0.10302079743096655


In [ ]:
#ensemble lda
ensemble_lda_model = EnsembleLda(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

In [ ]:
# Print the Keyword in the 5 topics
print(ensemble_lda_model.print_topics())

[(0, '0.028*"access" + 0.018*"card" + 0.016*"could" + 0.016*"th_floor" + 0.016*"en" + 0.015*"access_card" + 0.014*"issue" + 0.014*"let_know" + 0.013*"lost_access" + 0.012*"lost"')]


In [ ]:
getModelAccuacy(ensemble_lda_model,corpus,data,"Label")

0.2

In [ ]:
c_v_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_ensemble_lda = c_v_coherence_model_ensemble_lda.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_ensemble_lda)


C_V Coherence Score:  0.5158562384081099


In [ ]:
u_mass_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_ensemble_lda = u_mass_coherence_model_ensemble_lda.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_ensemble_lda)


u_mass Coherence Score:  -3.1626473376908413


In [ ]:
c_uci_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_ensemble_lda = c_uci_coherence_model_ensemble_lda.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_ensemble_lda)


c_uci Coherence Score:  -1.033979957905106


In [ ]:
c_npmi_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_ensemble_lda = c_npmi_coherence_model_ensemble_lda.get_coherence()
print('\nc_npmi Coherence Score: ', c_npmi_coherence_ensemble_lda)


c_npmi Coherence Score:  0.031161199618615208
